# 01: Data Preprocessing

This notebook handles cleaning, missing values, outlier detection, and aggregation of NPS and revenue data from healthcare clinics.

**Dataset**: 27,000+ NPS survey responses across 36 months (2022-2025)  
**Compliance**: Fully anonymized and LGPD/GDPR compliant

## Research Question
Does NPS truly predict revenue in healthcare clinics? This preprocessing step prepares the data for rigorous statistical testing.

In [1]:
## Import Required Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
print("Libraries imported successfully")

Libraries imported successfully


## Load Healthcare Dataset

In [6]:
# Load NPS and Revenue data
nps_data = pd.read_csv('data/raw/nps/nps_responses.csv')
revenue_data = pd.read_csv('data/raw/sales/revenue_by_clinic.csv')

print('NPS Dataset Shape:', nps_data.shape)
print('\nNPS Dataset Info:')
print(nps_data.info())
print('\nFirst few NPS records:')
print(nps_data.head())
print('\n' + '='*60)
print('Revenue Dataset Shape:', revenue_data.shape)
print('\nRevenue Dataset Info:')
print(revenue_data.info())
print('\nFirst few revenue records:')
print(revenue_data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/nps/nps_responses.csv'

## Handle Missing Values

In healthcare data, missing values require careful handling. We use statistical methods:
- **Continuous variables**: Median imputation (robust to outliers)
- **Time series**: Forward/backward fill for revenue
- **Categorical**: Mode or 'Unknown' flag

## Data Type Conversion

Ensure all columns have appropriate data types for analysis.

In [ ]:
# Convert date columns to datetime
date_col = 'date' if 'date' in nps_data.columns else 'Date'
if date_col in nps_data.columns:
    nps_data[date_col] = pd.to_datetime(nps_data[date_col])
    
date_col_rev = 'date' if 'date' in revenue_data.columns else 'Date'
if date_col_rev in revenue_data.columns:
    revenue_data[date_col_rev] = pd.to_datetime(revenue_data[date_col_rev])

# Ensure NPS scores are numeric
if 'nps_score' in nps_data.columns:
    nps_data['nps_score'] = pd.to_numeric(nps_data['nps_score'], errors='coerce')

# Ensure revenue is numeric
if 'revenue' in revenue_data.columns:
    revenue_data['revenue'] = pd.to_numeric(revenue_data['revenue'], errors='coerce')

print('Data types after conversion:')
print('\nNPS Data:')
print(nps_data.dtypes)
print('\nRevenue Data:')
print(revenue_data.dtypes)

## Outlier Detection and Treatment

Using statistical methods (IQR for NPS, Z-score for revenue) to identify anomalies without removing data—healthcare data is often sparse and valuable.

In [ ]:
# Detect outliers in NPS scores using IQR method
if 'nps_score' in nps_data.columns:
    Q1 = nps_data['nps_score'].quantile(0.25)
    Q3 = nps_data['nps_score'].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_nps = nps_data[(nps_data['nps_score'] < lower_bound) | (nps_data['nps_score'] > upper_bound)]
    print(f'NPS Outliers detected: {len(outliers_nps)} out of {len(nps_data)} ({100*len(outliers_nps)/len(nps_data):.2f}%)')
    print(f'Bounds: [{lower_bound:.2f}, {upper_bound:.2f}]')
    
    # Mark outliers rather than remove them
    nps_data['nps_is_outlier'] = (nps_data['nps_score'] < lower_bound) | (nps_data['nps_score'] > upper_bound)

# Detect outliers in Revenue using Z-score method
if 'revenue' in revenue_data.columns:
    z_scores = np.abs(stats.zscore(revenue_data['revenue'].dropna()))
    threshold = 3
    revenue_data['revenue_zscore'] = np.abs(stats.zscore(revenue_data['revenue'].fillna(revenue_data['revenue'].mean())))
    outliers_revenue = revenue_data[revenue_data['revenue_zscore'] > threshold]
    print(f'\nRevenue Outliers detected (Z-score > {threshold}): {len(outliers_revenue)} out of {len(revenue_data)} ({100*len(outliers_revenue)/len(revenue_data):.2f}%)')
    
    # Mark outliers
    revenue_data['revenue_is_outlier'] = revenue_data['revenue_zscore'] > threshold

## Aggregation by Clinic and Month

Convert individual survey responses into clinic-month aggregates with summary statistics.

In [ ]:
# Extract year-month from date columns
date_col = 'date' if 'date' in nps_data.columns else 'Date'
nps_data['year_month'] = nps_data[date_col].dt.to_period('M')

date_col_rev = 'date' if 'date' in revenue_data.columns else 'Date'
revenue_data['year_month'] = revenue_data[date_col_rev].dt.to_period('M')

# Aggregate NPS by clinic and month
clinic_col = 'clinic_id' if 'clinic_id' in nps_data.columns else 'clinic'
nps_agg = nps_data.groupby([clinic_col, 'year_month']).agg({
    'nps_score': ['mean', 'median', 'std', 'count'],
    'nps_is_outlier': 'sum' if 'nps_is_outlier' in nps_data.columns else 'count',
    date_col: 'first'
}).reset_index()

nps_agg.columns = ['clinic_id', 'year_month', 'nps_mean', 'nps_median', 'nps_std', 'nps_responses', 'nps_outliers', 'date']

print('Aggregated NPS Data (by clinic-month):')
print(nps_agg.head(10))
print(f'\nShape: {nps_agg.shape}')
print(f'\nClinics covered: {nps_agg["clinic_id"].nunique()}')
print(f'Months covered: {nps_agg["year_month"].nunique()}')

In [ ]:
# Aggregate Revenue by clinic and month
clinic_col_rev = 'clinic_id' if 'clinic_id' in revenue_data.columns else 'clinic'
revenue_agg = revenue_data.groupby([clinic_col_rev, 'year_month']).agg({
    'revenue': ['sum', 'mean', 'std'],
    'revenue_is_outlier': 'sum' if 'revenue_is_outlier' in revenue_data.columns else 'count',
    date_col_rev: 'first'
}).reset_index()

revenue_agg.columns = ['clinic_id', 'year_month', 'revenue_total', 'revenue_mean', 'revenue_std', 'revenue_outliers', 'date']

print('Aggregated Revenue Data (by clinic-month):')
print(revenue_agg.head(10))
print(f'\nShape: {revenue_agg.shape}')
print(f'Clinics covered: {revenue_agg["clinic_id"].nunique()}')
print(f'Months covered: {revenue_agg["year_month"].nunique()}')

## Create Derived Metrics

Calculate month-over-month changes and lagged features for regression analysis.

In [ ]:
# Merge NPS and Revenue data
combined = nps_agg.merge(revenue_agg, on=['clinic_id', 'year_month'], how='inner', suffixes=('_nps', '_revenue'))

# Sort by clinic and date
combined = combined.sort_values(['clinic_id', 'date_nps'])

# Calculate month-over-month changes
combined['nps_change'] = combined.groupby('clinic_id')['nps_mean'].pct_change() * 100
combined['revenue_change'] = combined.groupby('clinic_id')['revenue_total'].pct_change() * 100

# Calculate lagged NPS (NPS from previous month)
combined['nps_lag1'] = combined.groupby('clinic_id')['nps_mean'].shift(1)
combined['nps_lag3'] = combined.groupby('clinic_id')['nps_mean'].shift(3)

# Calculate revenue_per_respondent (efficiency metric)
combined['revenue_per_response'] = combined['revenue_total'] / combined['nps_responses']

print('Combined and enriched dataset:')
print(combined.head(15))
print(f'\nShape: {combined.shape}')
print(f'Columns: {combined.columns.tolist()}')

## Data Validation and Quality Checks

Final validation before analysis: check for duplicates, logical consistency, and summary statistics.

In [ ]:
# Remove duplicates
before_dedup = len(combined)
combined = combined.drop_duplicates(subset=['clinic_id', 'year_month'])
after_dedup = len(combined)
print(f'Duplicates removed: {before_dedup - after_dedup}')

# Logical consistency checks
print('\nLogical Consistency Checks:')
print(f'NPS scores in range [0, 100]: {combined["nps_mean"].between(0, 100).sum()}/{len(combined)}')
print(f'Revenue values positive: {(combined["revenue_total"] > 0).sum()}/{len(combined)}')
print(f'Rows with valid lagged NPS: {combined["nps_lag1"].notna().sum()}/{len(combined)}')

print('\n' + '='*60)
print('Summary Statistics:')
summary_cols = ['nps_mean', 'nps_std', 'revenue_total', 'nps_change', 'revenue_change', 'revenue_per_response']
print(combined[summary_cols].describe())

In [ ]:
# Save processed data
combined.to_csv('../data/processed/combined_nps_revenue.csv', index=False)
print('✓ Processed data saved to: ../data/processed/combined_nps_revenue.csv')
print(f'✓ Final dataset: {combined.shape[0]} clinic-months, {combined.shape[1]} features')
print(f'✓ Time span: {combined["year_month"].min()} to {combined["year_month"].max()}')

In [ ]:
# Identify missing values
print('Missing Values in NPS Data:')
missing_nps = nps_data.isnull().sum()
print(missing_nps[missing_nps > 0] if missing_nps.sum() > 0 else 'No missing values')
print(f'Total missing: {missing_nps.sum()}')

print('\n' + '='*60)
print('Missing Values in Revenue Data:')
missing_revenue = revenue_data.isnull().sum()
print(missing_revenue[missing_revenue > 0] if missing_revenue.sum() > 0 else 'No missing values')
print(f'Total missing: {missing_revenue.sum()}')

# Handle missing values in NPS data
if 'nps_score' in nps_data.columns:
    nps_data['nps_score'].fillna(nps_data['nps_score'].median(), inplace=True)

# For categorical data: use mode or 'Unknown'
for col in nps_data.select_dtypes(include='object').columns:
    if nps_data[col].isnull().any():
        nps_data[col].fillna(nps_data[col].mode()[0] if len(nps_data[col].mode()) > 0 else 'Unknown', inplace=True)

# Handle missing values in Revenue data - sort first, then fill
revenue_data = revenue_data.sort_values('date' if 'date' in revenue_data.columns else 'Date')
if 'revenue' in revenue_data.columns:
    revenue_data['revenue'].fillna(method='ffill', inplace=True)
    revenue_data['revenue'].fillna(method='bfill', inplace=True)

print('\nMissing values after imputation:')
print(f'NPS: {nps_data.isnull().sum().sum()}')
print(f'Revenue: {revenue_data.isnull().sum().sum()}')

(654, 4)
(104, 4)
